<a href="https://colab.research.google.com/github/ehabsuliman/Essay-Grading-Transformer/blob/main/Grading_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is a prompt and a partial response. Complete the response appropriately.

### Prompt:
{}

### Response:
{}"""


In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    prompts    = examples["prompt"]
    completions = examples["completion"]
    texts = []
    for prompt, completion in zip(prompts, completions):
        text = alpaca_prompt.format(prompt, completion) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }


In [ ]:
from datasets import load_dataset

# Load your JSONL file
dataset = load_dataset("json", data_files="/content/NLP_exam_score.jsonl", split="train")

# Format the prompts using your custom function
dataset = dataset.map(formatting_prompts_func, batched=True)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.373400
2,2.435500
3,2.173300
4,1.684000
5,1.345000
6,1.161400
7,0.987400
8,0.859600
9,0.743400
10,0.730600


In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.883 GB of memory reserved.


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

440.1626 seconds used for training.
7.34 minutes used for training.
Peak reserved memory = 6.883 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 46.693 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
# Save model and tokenizer
save_path = "/content/drive/MyDrive/model_saved"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/model_saved/tokenizer_config.json',
 '/content/drive/MyDrive/model_saved/special_tokens_map.json',
 '/content/drive/MyDrive/model_saved/tokenizer.model',
 '/content/drive/MyDrive/model_saved/added_tokens.json',
 '/content/drive/MyDrive/model_saved/tokenizer.json')

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/model_saved",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
model.eval()


==((====))==  Unsloth 2025.5.7: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load /content/drive/MyDrive/model_saved as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [ ]:
import torch

prompt = """Below is a prompt and a partial response. Complete the response appropriately.

### Prompt:
Question: Explain overfittng in machine learning?
Reference Answer: Overfitting occurs when a machine learning model learns the training data too well,
                  including its noise and outliers, resulting in poor generalization to new data.
                  The model performs well on the training set but poorly on unseen data
Student Answer: overfitting is when the model preforms very well on the training data and dont do well on the unseen test data
   it dont generalize well like it remember the data but dont understand it
Mark Scheme: Defines overfitting clearly
             Mentions memorization or learning noise
             Addresses poor generalization
             Provides a contrast between training and test performance

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Below is a prompt and a partial response. Complete the response appropriately.

### Prompt:
Question: Explain overfittng in machine learning?
Reference Answer: Overfitting occurs when a machine learning model learns the training data too well,
                  including its noise and outliers, resulting in poor generalization to new data.
                  The model performs well on the training set but poorly on unseen data
Student Answer: overfitting is when the model preforms very well on the training data and dont do well on the unseen test data
   it dont generalize well like it remember the data but dont understand it
Mark Scheme: Defines overfitting clearly
             Mentions memorization or learning noise
             Addresses poor generalization
             Provides a contrast between training and test performance

### Response:
Score:

### Response:
 75
Rationale: Good but lacks clarity on how it harms performance.


In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/model_saved",  # path to saved model
    load_in_4bit = True,     # keep this consistent
    dtype = None,            # auto-detect
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.8: Fast Mistral patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

Unsloth: Will load /content/drive/MyDrive/model_saved as a legacy tokenizer.
Unsloth 2025.5.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
import json
import torch

all_outputs = []

# Process each line (each prompt) in the JSONL file
with open("/content/converted_data.jsonl", "r") as f:
    for line in f:
        entry = json.loads(line)
        prompt = entry["prompt"]

        # Tokenize and generate
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                eos_token_id=tokenizer.eos_token_id,
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append both prompt and response to output list
        all_outputs.append({
            "prompt": prompt,
            "response": response.strip()
        })

# Save all outputs to a new JSON file
with open("/content/generated_responses.json", "w") as out_file:
    json.dump(all_outputs, out_file, ensure_ascii=False, indent=2)

print("All responses generated and saved to generated_responses.json")


All responses generated and saved to generated_responses.json


In [9]:
import torch

prompt = """Below is a prompt and a partial response. Complete the response appropriately.

### Prompt:
Question: What is the role of game playing in ai in modern artificial intelligence systems?
Reference Answer: Game Playing in AI plays a vital role in AI by providing techniques and tools to solve domain-specific problems. It enables systems to perform tasks such as reasoning, planning, interaction, or adaptation depending on the context of the topic.
It enables systems to perform tasks such as reasoning, planning, interaction, or adaptation depending on the context of the topic.
Student Answer: game playing in ai is when AI works like a computer game or hardware
Mark Scheme: 1.Defines game playing in ai
          2. Mentions key function in AI
          3. Provides a relevant application or example
          4. Connects to AI goals like automation or decision-making
### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Below is a prompt and a partial response. Complete the response appropriately.

### Prompt:
Question: What is the role of game playing in ai in modern artificial intelligence systems?
Reference Answer: Game Playing in AI plays a vital role in AI by providing techniques and tools to solve domain-specific problems. It enables systems to perform tasks such as reasoning, planning, interaction, or adaptation depending on the context of the topic. 
It enables systems to perform tasks such as reasoning, planning, interaction, or adaptation depending on the context of the topic.
Student Answer: game playing in ai is when AI works like a computer game or hardware
Mark Scheme: 1.Defines game playing in ai
          2. Mentions key function in AI
          3. Provides a relevant application or example
          4. Connects to AI goals like automation or decision-making
### Response:
Score:

### Response:
 0
Rationale: Incorrectly focuses on hardware and gaming rather than AI applications.


In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [13]:
import pandas as pd

# Load the JSON data, assuming it's JSON Lines format
df = pd.read_json('/content/AI_Topics_Rationales_Scores_Evaluate.json', lines=True)
print(df.columns)

Index(['predicted_score', 'generated_rationale', 'true_score',
       'true_rationale'],
      dtype='object')


In [14]:
import pandas as pd

# Load the JSON data
df = pd.read_json('/content/AI_Topics_Rationales_Scores_Evaluate.json', lines=True)

# Required columns
required_cols = ["generated_rationale", "true_rationale", "predicted_score", "true_score"]

# Verify columns
assert all(col in df.columns for col in required_cols), "Missing required columns."
print("All required columns are present:", df.columns.tolist())

All required columns are present: ['predicted_score', 'generated_rationale', 'true_score', 'true_rationale']


In [15]:
missing_data = df[required_cols].isnull().any(axis=1)
if missing_data.any():
    print("Rows with missing data:")
    print(df[missing_data])
else:
    print("No missing data in required columns.")

No missing data in required columns.


In [17]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ----------------------
# Load Your Dataset
# ----------------------
file_path = "/content/AI_Topics_Rationales_Scores_Evaluate.json"

# Load JSON Lines data
try:
    df = pd.read_json(file_path, lines=True)
except ValueError as e:
    print(f"Error loading JSON: {e}")
    exit(1)

# Required columns
required_cols = ["generated_rationale", "true_rationale", "predicted_score", "true_score"]
assert all(col in df.columns for col in required_cols), "Missing required columns."
print("All required columns are present:", df.columns.tolist())

# Drop rows with missing data
df.dropna(subset=required_cols, inplace=True)
print(f"Number of valid rows after dropping missing data: {len(df)}")

# ----------------------
# Load SBERT Model
# ----------------------
try:
    model = SentenceTransformer("all-MiniLM-L6-v2")
except Exception as e:
    print(f"Error loading SentenceTransformer model: {e}")
    exit(1)

# ----------------------
# Embed Rationales
# ----------------------
gen_rationales = df["generated_rationale"].tolist()
true_rationales = df["true_rationale"].tolist()

# Ensure no empty strings in rationales
if not all(gen_rationales) or not all(true_rationales):
    print("Warning: Empty rationale(s) found. Replacing with placeholder.")
    gen_rationales = [r if r.strip() else "No rationale provided" for r in gen_rationales]
    true_rationales = [r if r.strip() else "No rationale provided" for r in true_rationales]

gen_embeds = model.encode(gen_rationales, convert_to_tensor=True)
true_embeds = model.encode(true_rationales, convert_to_tensor=True)

# Compute cosine similarity for each pair
cosine_scores = util.cos_sim(gen_embeds, true_embeds)
pairwise_similarities = cosine_scores.diag().cpu().numpy()

# ----------------------
# Score Metrics
# ----------------------
predicted_scores = df["predicted_score"].tolist()
true_scores = df["true_score"].tolist()

mae = mean_absolute_error(true_scores, predicted_scores)
rmse = mean_squared_error(true_scores, predicted_scores) ** 0.5
pearson, _ = pearsonr(true_scores, predicted_scores)
spearman, _ = spearmanr(true_scores, predicted_scores)

print(f"📊 Score Evaluation Metrics:")
print(f" - MAE:     {mae:.4f}")
print(f" - RMSE:    {rmse:.4f}")
print(f" - Pearson: {pearson:.4f}")
print(f" - Spearman:{spearman:.4f}")

# ----------------------
# Embedding Similarity Metrics
# ----------------------
mean_similarity = np.mean(pairwise_similarities)
print(f"\n🧠 Mean Rationale Embedding Similarity: {mean_similarity:.4f}")

# ----------------------
# Correlation Between Rationale Quality and Score Error
# ----------------------
score_errors = np.abs(np.array(predicted_scores) - np.array(true_scores))
sim_vs_score_corr, _ = spearmanr(score_errors, pairwise_similarities)

print(f"\n🔁 Spearman Correlation (Rationale Similarity vs. Score Error): {sim_vs_score_corr:.4f}")

All required columns are present: ['predicted_score', 'generated_rationale', 'true_score', 'true_rationale']
Number of valid rows after dropping missing data: 75
📊 Score Evaluation Metrics:
 - MAE:     20.5333
 - RMSE:    27.9762
 - Pearson: 0.7285
 - Spearman:0.7407

🧠 Mean Rationale Embedding Similarity: 0.4279

🔁 Spearman Correlation (Rationale Similarity vs. Score Error): -0.0477
